In [2]:
#function to do all of the below futures preprocessing work
import pandas as pd
import math


#functions to clean and resample futures data

import numpy as np

def is_valid_contract(symbol: str) -> bool:
    """Return True if symbol looks like a single futures contract (e.g. GCZ2), not a spread like GCZ2-GCG3."""
    return '-' not in symbol and symbol.startswith("GC") and len(symbol) in (4, 5)

def load_and_resample_futures_data(csv_path: str, resample_freq='1min'):
    """
    Load gold futures data, filter out spreads, and resample prices to specified frequency.
    
    Parameters:
        csv_path (str): Path to CSV file
        resample_freq (str): e.g. '1min', '5min'

    Returns:
        pd.DataFrame: Resampled DataFrame with datetime index and one column per contract
    """
    # Load the CSV file
    df = pd.read_csv(csv_path)

    # Use ts_event as the real timestamp if available, else ts_recv
    try:
        df['timestamp'] = pd.to_datetime(df['ts_event'].fillna(df['ts_recv']))
    except:
        print(csv_path)
        exit(0)
    df = df.set_index('timestamp')

    # Filter for only valid individual futures contracts
    df = df[df['symbol'].apply(is_valid_contract)]

    # Keep only rows with valid price values
    df = df[pd.to_numeric(df['price'], errors='coerce').notnull()]
    df['price'] = df['price'].astype(float)

    # Pivot to create a contract × time matrix
    futures_dict = {}
    for symbol, group in df.groupby('symbol'):
        resampled = group['price'].resample(resample_freq).last().dropna()
        futures_dict[symbol] = resampled

    # Combine into a single DataFrame
    futures_df = pd.DataFrame(futures_dict)
    futures_df = futures_df.sort_index()

    return futures_df


    




def process_futures_file(file_name: str) -> pd.DataFrame:

    #load df
    df = load_and_resample_futures_data(file_name)

    tickers = {}
    for row in df.iterrows():
        for ticker in row[1].keys():
            if ticker not in tickers.keys(): tickers[ticker] = 0
            val = row[1][ticker]
            if not math.isnan(val): tickers[ticker] += 1

    ticker_liquidity = sorted(tickers.items(), reverse=True, key=lambda x:x[1])


    #could be a holiday
    if not len(ticker_liquidity):
        return None
    most_liquid_ticker = ticker_liquidity[0]

    most_liquid_ticker = most_liquid_ticker[0]



    price_col = 'price_' + most_liquid_ticker
    liquid_df = {'timestamp':[],price_col:[]}
    for row in df.iterrows():
        if not math.isnan(row[1][most_liquid_ticker]):
            liquid_df['timestamp'].append(row[0])
            liquid_df[price_col].append(row[1][most_liquid_ticker])

    futures_df = pd.DataFrame(liquid_df)

    return futures_df

In [3]:
import os
import pandas as pd
from tqdm import tqdm

processed_dfs = []

directory = '../../../data/futures_data/GLBX/'

# Wrap the file list with tqdm for a progress bar
for filename in tqdm(os.listdir(directory), desc="Processing CSV files"):
    if filename.endswith('.csv') and 'glbx' in filename:
        full_path = os.path.join(directory, filename)
        df = process_futures_file(full_path)
        if df is not None:
            processed_dfs.append(df)

# Concatenate all processed DataFrames into one
final_df = pd.concat(processed_dfs, ignore_index=True)



Processing CSV files:   0%|          | 0/3130 [00:00<?, ?it/s]/var/folders/lh/1dgly4594bn8mdjfz3vmlyr80000gn/T/ipykernel_87573/3626958332.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['ts_event'].fillna(df['ts_recv']))
Processing CSV files:   7%|▋         | 209/3130 [00:15<02:38, 18.45it/s]/var/folders/lh/1dgly4594bn8mdjfz3vmlyr80000gn/T/ipykernel_87573/3626958332.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['ts_event'].fillna(df['ts_recv']))
Processing CSV files:  28%|██▊       | 873/3130 [01:06<02:50, 13.24it/s]/var/folders/lh/1dgly4594bn8mdjfz3vmlyr80000gn/T/ipykernel_87573/3626958332.py:30: UserWarning: Could not infer fo

In [4]:
final_df.to_csv('futures_data.csv')

In [8]:
# Drop index column if present
df = final_df
if df.columns[0] == '':
    df = df.drop(columns=[''])

# Melt to long format
long_df = df.melt(id_vars=['timestamp'], var_name='symbol', value_name='price')

# Drop rows with missing prices
long_df = long_df.dropna(subset=['price'])

# Clean symbol names
long_df['symbol'] = long_df['symbol'].str.replace('price_', '')
print(long_df['timestamp'].dtype)
long_df['timestamp'] = pd.to_datetime(long_df['timestamp'], utc=True)
print(long_df['timestamp'].dtype)
# Save to CSV
long_df.to_csv('futures_data_processed.csv', index=False)

datetime64[ns, UTC]
datetime64[ns, UTC]
